#### Zadanie:

Doplňte cesty do očisteného logovacieho súboru, v ktorom boli identifikovaný používatelia a sedenia pomocou metódy STT_Q.

Odovzdajte zdrojový kód programu + logovací súbor doplnený o identifikáciu používateľov (v jednom zip).

In [1]:
import pandas as pd
from urllib.parse import urlparse

In [2]:
file = pd.read_csv("12_Identifikácia_sedení_SLength.csv", escapechar = ";", encoding ="utf-8", index_col = False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# I will use here www.example.com, but it referes to your domain name, which access logs are being examined
# (case specific)

# we need to create same structure of the data for the one we are searching 
# there are 2 ways. Either transform Request by adding https://www.example.com to each or delete
# the https://www.example.com from the referer. This way we will normalize them and can compare at latter instance

# one more rule is added., After trial and error I found that referer is one of 3 which can be treated as same:
# 1. https://www.example.com/
# 2. https://example.com
# 3. http://www.example.com

new_referer = []
for refer in file['Referer']:
    if (pd.isna(refer) or not refer.startswith("https://www.example.com/")
    or not refer.startswith("https://example.com/")
    or not refer.startswith("http://www.example.com/")):
        new_referer.append(refer)
    else:        
        referer_path = urlparse(refer).path + urlparse(refer).params + urlparse(refer).query + urlparse(refer).fragment
        new_referer.append(referer_path)


In [4]:
file['Referer'] = new_referer

In [5]:
df_STT_Q = file

rows = df_STT_Q.iloc[0:0]
STT_Q = 0
df_temp = df_STT_Q.iloc[0:0]

for index, row in df_STT_Q.iterrows():
    if(row.STT_Q == STT_Q):
        if(len(rows) and (rows['Request Uri'].iloc[-1] == row['Referer'])):
            rows = rows.append(row,ignore_index=True)
        elif((row['Request Uri'] in list(rows['Request Uri'])) and (not pd.isna(row['Referer'])) and str(row['Referer'].startswith('/'))):
            last_index = rows['Request Uri'].where(rows['Request Uri']==row['Request Uri']).last_valid_index()
            if (rows['Referer'].iloc[last_index] == row['Referer']):
                part_refer = list(rows['Request Uri'].iloc[last_index:])
                part_refer.reverse()
                part_url = list(rows['Referer'].iloc[last_index:])
                part_url.reverse()
                part_tuples = list(zip(part_url,part_refer))
                for item in part_tuples:
                    new_row = rows.iloc[-1]
                    new_row['Request Uri'] = item[0]
                    new_row['Referer'] = item[1]
                    new_row['UNIXTIME'] += 1
                    new_row.length = 1
                    df_temp = df_temp.append(new_row,ignore_index=True)
                rows = df_STT_Q.iloc[0:0]
            else:
                rows = df_STT_Q.iloc[0:0]
                try:
                    if str(row['Referer'].startswith('/')):
                        rows = rows.append(row,ignore_index=True)
                except:
                    continue
        else:
            rows = df_STT_Q.iloc[0:0]
            try:
                if str(row['Referer'].startswith('/')):
                    rows = rows.append(row,ignore_index=True)
            except:
                continue
    else:      
        STT_Q = row.STT_Q
        rows = df_STT_Q.iloc[0:0]
        try:
            if str(row['Referer'].startswith('/')):
                rows = rows.append(row,ignore_index=True)
        except:
            continue

<ipython-input-5-dbe7fb7fd245>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['Request Uri'] = item[0]
<ipython-input-5-dbe7fb7fd245>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['Referer'] = item[1]
<ipython-input-5-dbe7fb7fd245>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['UNIXTIME'] += 1


In [6]:
#df_temp.cookie = 'new log'
df_STT_Q = df_STT_Q.append(df_temp,ignore_index=True)
df_STT_Q.sort_values(by=['Remote Host', 'User-Agent', 'UNIXTIME'],ascending=[True, True, True], inplace=True)
df_STT_Q

,USER_ID,SEQUENCE_ID,Remote Host,Remote Logname,Remote User,Request Time,UNIXTIME,Length,RLength,STT_mean,STT_Q,SLength,Request Method,Request Uri,Request Protocol,Final Status Code,Bytes Sent,Referer,User-Agent
0,1,1,1.0.133.100,NaN,NaN,2021-08-15 01:21:11+02:00,1.628983e+09,NaN,1,1,1,1,GET,/,HTTP/1.0,200.0,10466.0,http://www.almhuette-raith.at,Mozilla/5.0 (Windows NT 6.1 Win64 x64) AppleWe...
1,2,2,1.0.145.250,NaN,NaN,2021-05-18 02:41:17+02:00,1.621298e+09,NaN,2,2,2,2,GET,/,HTTP/1.0,200.0,10466.0,http://www.almhuette-raith.at,Mozilla/5.0 (Windows NT 6.0 Win64 x64) AppleWe...
2,3,3,1.0.220.38,NaN,NaN,2021-03-01 15:01:20+01:00,1.614607e+09,NaN,3,3,3,3,GET,/,HTTP/1.0,200.0,10466.0,http://www.almhuette-raith.at,Mozilla/5.0 (Windows NT 10.0 Win64 x64) AppleW...
3,4,4,1.0.238.210,NaN,NaN,2020-12-24 22:03:52+01:00,1.608844e+09,NaN,4,4,4,4,GET,/administrator/index.php,HTTP/1.1,200.0,4263.0,NaN,Mozilla/4.0 (compatible MSIE 6.0 Windows NT 5....
4,5,5,1.1.185.155,NaN,NaN,2021-02-26 19:02:17+01:00,1.614363e+09,NaN,5,5,5,5,GET,/,HTTP/1.0,200.0,10466.0,http://www.almhuette-raith.at,Mozilla/5.0 (Windows NT 10.0 Win64 x64) AppleW...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37020,19572,24972,99.42.33.227,NaN,NaN,2021-01-22 03:35:06+01:00,1.611283e+09,30.0,26449,25819,27311,25383,GET,/,HTTP/1.1,200.0,10479.0,http://www.almhuette-raith.at/apache-log/,Mozilla/5.0 (Macintosh Intel Mac OS X 10_15_7)...
37021,19572,24972,99.42.33.227,NaN,NaN,2021-01-22 03:35:36+01:00,1.611283e+09,10.0,26449,25819,27312,25383,GET,/index.php?option=com_content&view=article&id=...,HTTP/1.1,200.0,7936.0,http://www.almhuette-raith.at/,Mozilla/5.0 (Macintosh Intel Mac OS X 10_15_7)...
37022,19572,24972,99.42.33.227,NaN,NaN,2021-01-22 03:35:46+01:00,1.611283e+09,3.0,26449,25819,27312,25383,GET,/index.php?option=com_content&view=article&id=...,HTTP/1.1,200.0,7991.0,http://www.almhuette-raith.at/index.php?option...,Mozilla/5.0 (Macintosh Intel Mac OS X 10_15_7)...
37023,19572,24972,99.42.33.227,NaN,NaN,2021-01-22 03:35:49+01:00,1.611283e+09,9.0,26449,25819,27312,25383,GET,/index.php?option=com_phocagallery&view=catego...,HTTP/1.1,200.0,32583.0,http://www.almhuette-raith.at/index.php?option...,Mozilla/5.0 (Macintosh Intel Mac OS X 10_15_7)...


In [7]:
# writing the file to new csv, escape char ";" to not get User-Agent divided on export/write
file.to_csv("14_Dopĺňanie_ciest_STT_Q.csv", encoding="utf-8",  escapechar=";", index=False)